Here we are basicly replicating my [Human Protein Atlas coexpression](https://github.com/TomConlin/human_protein_atlas_coexpression) notebook but with BGEE expression data

In [1]:
import csv
import re
#import array
import numpy as np
import matplotlib.pyplot as plt
import statistics
import requests
import json
import yaml
import re
import os
import networkx as nx
import datetime
import operator

In [2]:

# reduce ordered catagories to numbers 
# if level is unreported default to zero
levels = {
    'absent' : -1,
    'present': 1
}

reliabilities = {
    'gold quality' : 2,
    'silver quality' : 1,
    'bronze quality' : 0
}

In [3]:
a = [1,1,1,0,0,0,-1,-1,-1]
b = [1,0,-1,1,0,-1,1,0,-1]

for i in range(9):
    print(a[i],b[i],a[i] * b[i])

1 1 1
1 0 0
1 -1 -1
0 1 0
0 0 0
0 -1 0
-1 1 -1
-1 0 0
-1 -1 1


Note that:  
 - agreement positive or negative add
 - disagreement postive or negative subtract 
 - any comparison with unreported stays unreported

In [4]:
for i in range(9):
    print(a[i],b[i],a[i] & b[i])

1 1 1
1 0 0
1 -1 1
0 1 0
0 0 0
0 -1 0
-1 1 1
-1 0 0
-1 -1 -1


note:  logical AND 
 - inconsistant capture of concordancce
 - sufficent capture of reported-ness
 - cheaper than multiply (should be)

Nomenclature maintaince has to happen somewhere  
Either early or late.  
Although Ensembl is very lilely to be current     
Monarch tends to standardise on NCBIGene IDs for other things 

[HGNC API doc](https://www.genenames.org/help/rest-web-service-help)  
and  
[Bulk Download](https://www.genenames.org/cgi-bin/download)

in this process, the time to convert ids and symbols is after
the set of tissue is associated with a gene  
b/w then it is once per profile, instead of per measurement.

In [5]:
# https://www.genenames.org/cgi-bin/download?
# col=gd_app_sym
# col=gd_pub_eg_id
# col=gd_pub_ensembl_id
# col=md_eg_id&
# col=md_ensembl_id
# status=Approved
# status_opt=2
# order_by=gd_pub_eg_id
# format=text
# submit=submit

HGNCURL='http://www.genenames.org/cgi-bin/download?'

query = "&".join([
    'col=gd_pub_ensembl_id',
    'col=md_ensembl_id',
    'col=gd_pub_eg_id',
    'col=md_eg_id',
    'col=gd_app_sym',
    'order_by=gd_pub_ensembl_id',
    'status=Approved',
    'status_opt=2',
    'format=text',
    'submit=submit'
])

In [6]:
response = requests.get(HGNCURL + query)

In [7]:
if response.status_code == requests.codes.ok:
    hgnc = response.text.splitlines()
    hgnc.pop(0)  # header
else: 
    print('ERROR ' + response.url + ' returned '+ str(response.status_code))

In [8]:
ensg_sym = {}  # in case there are any not mapped to ncbi as well
sym_ensg = {}
ensg_ncbi = {}
ncbi_sym = {}

sym_ncbi = {}

for line in hgnc:
    (ens1, ens2, eg1, eg2, symbol) = re.split(r'\t',line)
    ensg = ens2 if ens1 == '' else ens1
    ncbi = eg2 if eg1 == '' else eg1    
    if ensg != '': 
        if symbol != '':
            ensg_sym[ensg] = symbol
            sym_ensg[symbol] = ensg # to search network for our FA genes
        if ncbi != '':
            ensg_ncbi[ensg] = ncbi
    if ncbi != '':
        ncbi_sym[ncbi] = symbol
        sym_ncbi[symbol] = ncbi

In [9]:
len(ensg_sym)

36864

In [10]:
len(ncbi_sym) 

41152

In [11]:
len(ensg_ncbi)

36856

As feared thare are tens of thousands of these measurements won't map directly back to something we can integreate, but it is still important to process them all because if something comes up as interesting, one could at least go back to the underlying sequence.  

note:  
we can (should) never get an empty gene symbol from the nomenclature athority

as there are slightly more ensembl->symbol than ensembl->ncbi  
 - first try ensembl->symbol   
 - then ensembl->ncbi->symbol 

In [12]:
with open('ensgid_symbol_hgnc.yaml', 'w') as fh:
    yaml.dump(ensg_sym, fh)
with open('ncbi_symbol_hgnc.yaml', 'w') as fh:
    yaml.dump(ncbi_sym, fh)    
with open('ensg_ncbi.yaml', 'w') as fh:
    yaml.dump(ensg_ncbi, fh) 

Use the tissue measurement frequency to set the order for tissue profile recording

In [13]:
file = 'tissue_count.tab' 
# count tissue_id
tissue_index = {}
with open(file, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter='\t')
    #csvreader.__next__()  # header
    line_number = 0
    for line in csvreader:
        (count, tissue_id) = line
        tissue_index[tissue_id] = line_number
        line_number += 1
tissue_length = len(tissue_index)        

In [14]:
file = 'Homo_sapiens_expr_simple.tsv'
#1:Gene ID
#2:"Gene name"
#3:Anatomical entity ID
#4:"Anatomical entity name"
#5:Expression
#6:Call quality
#7:Expression rank

# make places to stash stuff 
gene_tissue = {}
#gene_tissue_rank_curve = {}
tissue_names = {}
gene_names = {}

with open(file, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
    csvreader.__next__()  # header
    for line in csvreader:
        # print(line)
        (gene_id, gene_name, tissue_id, tissue_name, level, reliability, rank) = line
        
        # record the measurement   
        if gene_id in gene_tissue:
            gene_tissue[gene_id][tissue_index[tissue_id]] = levels[level]
        else:
            gene_tissue[gene_id] = np.array([0] * tissue_length)
            gene_tissue[gene_id][tissue_index[tissue_id]] = levels[level]

        # save human friendly tissue label 
        if tissue_id not in tissue_names:
            tissue_names[tissue_id] = tissue_name
        
        # save human friendly gene symbol 
        if gene_id not in gene_names:
            gene_names[gene_id] = gene_name

In [15]:
# update gene symbols
for gene_id in gene_tissue:
      # try to bring gene info up to date
        if gene_id in ensg_sym and gene_name != gene_names[gene_id]:
            print('NOTE: '+ ensg_sym[gene_id]+ ' is replacing ' + gene_names[gene_id] + ' b/c ' + gene_id)
            gene_names[gene_id] = ensg_sym[gene_id]
        elif gene_id in ensg_ncbi and ensg_ncbi[gene_id] in ncbi_sym \
                and  gene_names[gene_id] != ncbi_sym[ensg_ncbi[gene_id]]:
            print('NOTE: '+ ncbi_sym[ensg_ncbi[gene_id]] + ' is replacing ' + gene_name + ' b/c ' + gene_id + ' -> ' + ensg_ncbi[gene_id])
            gene_names[gene_id] = ncbi_sym[ensg_ncbi[gene_id]]


NOTE: ADAMTSL4 is replacing ADAMTSL4 b/c ENSG00000143382
NOTE: CHCHD2 is replacing CHCHD2 b/c ENSG00000106153
NOTE: CDX4 is replacing CDX4 b/c ENSG00000131264
NOTE: NME1-NME2 is replacing NME1-NME2 b/c ENSG00000011052
NOTE: STIP1P3 is replacing STIP1P3 b/c ENSG00000225536
NOTE: RNU6-246P is replacing RNU6-246P b/c ENSG00000207433
NOTE: HDLBP is replacing HDLBP b/c ENSG00000115677
NOTE: SOHLH2 is replacing SOHLH2 b/c ENSG00000120669
NOTE: HACL1 is replacing HACL1 b/c ENSG00000131373
NOTE: RNY1P10 is replacing RNY1P10 b/c ENSG00000200138
NOTE: PILRB is replacing PILRB b/c ENSG00000121716
NOTE: GPR119 is replacing GPR119 b/c ENSG00000147262
NOTE: KARSP2 is replacing KARSP2 b/c ENSG00000230371
NOTE: PHYHD1 is replacing PHYHD1 b/c ENSG00000175287
NOTE: IKZF4 is replacing IKZF4 b/c ENSG00000123411
NOTE: KRTAP13-3 is replacing KRTAP13-3 b/c ENSG00000240432
NOTE: MIR575 is replacing MIR575 b/c ENSG00000207746
NOTE: ITGA2 is replacing ITGA2 b/c ENSG00000164171
NOTE: MTATP6P23 is replacing MTATP

NOTE: FAM86B1 is replacing FAM86B1 b/c ENSG00000186523
NOTE: PRELID3BP11 is replacing PRELID3BP11 b/c ENSG00000231108
NOTE: RNU6-1284P is replacing RNU6-1284P b/c ENSG00000200571
NOTE: RPS4XP11 is replacing RPS4XP11 b/c ENSG00000234335
NOTE: PCDHA1 is replacing PCDHA1 b/c ENSG00000204970
NOTE: SMNDC1 is replacing SMNDC1 b/c ENSG00000119953
NOTE: MIR3178 is replacing MIR3178 b/c ENSG00000266232
NOTE: REREP1Y is replacing REREP1Y b/c ENSG00000225189
NOTE: CD300LD is replacing CD300LD b/c ENSG00000204345
NOTE: KIAA1755 is replacing KIAA1755 b/c ENSG00000149633
NOTE: C22orf42 is replacing C22orf42 b/c ENSG00000205856
NOTE: LCK is replacing LCK b/c ENSG00000182866
NOTE: KCTD10 is replacing KCTD10 b/c ENSG00000110906
NOTE: SLC25A3P1 is replacing SLC25A3P1 b/c ENSG00000236253
NOTE: LRTOMT is replacing LRTOMT b/c ENSG00000184154
NOTE: PITPNB is replacing PITPNB b/c ENSG00000180957
NOTE: DHRS7 is replacing DHRS7 b/c ENSG00000100612
NOTE: CICP5 is replacing CICP5 b/c ENSG00000233771
NOTE: PGR is

NOTE: RNU6-430P is replacing RNU6-430P b/c ENSG00000252887
NOTE: JCAD is replacing KIAA1462 b/c ENSG00000165757
NOTE: SMIM30 is replacing LINC00998 b/c ENSG00000214194
NOTE: UBE2V1P6 is replacing UBE2V1P6 b/c ENSG00000235995
NOTE: THUMPD3-AS1 is replacing THUMPD3-AS1 b/c ENSG00000206573
NOTE: SOX1-OT is replacing LINC00403 b/c ENSG00000224243
NOTE: LINC02318 is replacing RP11-1070N10.4 b/c ENSG00000258390
NOTE: TTC21A is replacing TTC21A b/c ENSG00000168026
NOTE: TRAV13-1 is replacing TRAV13-1 b/c ENSG00000211788
NOTE: MIR381HG is replacing MIR381HG b/c ENSG00000258861
NOTE: TXNL1 is replacing TXNL1 b/c ENSG00000091164
NOTE: RNU6-1005P is replacing RNU6-1005P b/c ENSG00000207248
NOTE: MTATP8P2 is replacing MTATP8P2 b/c ENSG00000229604
NOTE: CASR is replacing CASR b/c ENSG00000036828
NOTE: MIS12 is replacing MIS12 b/c ENSG00000167842
NOTE: ABHD17AP1 is replacing ABHD17AP1 b/c ENSG00000198658
NOTE: TEDC2 is replacing C16orf59 b/c ENSG00000162062
NOTE: SNORD56B is replacing SNORD56B b/c E

NOTE: ZNF536 is replacing ZNF536 b/c ENSG00000198597
NOTE: SMIM1 is replacing SMIM1 b/c ENSG00000235169
NOTE: FOXD4L6 is replacing FOXD4L6 b/c ENSG00000273514
NOTE: WWC2 is replacing WWC2 b/c ENSG00000151718
NOTE: DIRAS2 is replacing DIRAS2 b/c ENSG00000165023
NOTE: DUX4L10 is replacing DUX4L10 b/c ENSG00000278664
NOTE: MT1A is replacing MT1A b/c ENSG00000205362
NOTE: IGSF21 is replacing IGSF21 b/c ENSG00000117154
NOTE: TEKT1 is replacing TEKT1 b/c ENSG00000167858
NOTE: PRKCZ-AS1 is replacing RP11-181G12.2 b/c ENSG00000182873
NOTE: IGHV1OR15-4 is replacing IGHV1OR15-4 b/c ENSG00000270356
NOTE: FEM1B is replacing FEM1B b/c ENSG00000169018
NOTE: IGHV7-34-1 is replacing IGHV7-34-1 b/c ENSG00000253325
NOTE: ENOX1 is replacing ENOX1 b/c ENSG00000120658
NOTE: LINC02106 is replacing CTD-2089N3.2 b/c ENSG00000251573
NOTE: LINC01515 is replacing LINC01515 b/c ENSG00000228065
NOTE: ITPR1-AS1 is replacing ITPR1-AS1 b/c ENSG00000231249
NOTE: C21orf91-OT1 is replacing C21orf91-OT1 b/c ENSG000002407

NOTE: OXER1 is replacing OXER1 b/c ENSG00000162881
NOTE: IGHV3-22 is replacing IGHV3-22 b/c ENSG00000253957
NOTE: DUXAP6 is replacing DUXAP6 b/c ENSG00000258726
NOTE: SEC23IP is replacing SEC23IP b/c ENSG00000107651
NOTE: TMPRSS11E is replacing TMPRSS11E b/c ENSG00000087128
NOTE: RASAL2-AS1 is replacing RASAL2-AS1 b/c ENSG00000224687
NOTE: PEMT is replacing PEMT b/c ENSG00000133027
NOTE: ADGRF5P1 is replacing ADGRF5P1 b/c ENSG00000227582
NOTE: GNG4 is replacing GNG4 b/c ENSG00000168243
NOTE: CCDC177 is replacing CCDC177 b/c ENSG00000267909
NOTE: TRIM35 is replacing TRIM35 b/c ENSG00000104228
NOTE: PRR27 is replacing PRR27 b/c ENSG00000187533
NOTE: OR2M1P is replacing OR2M1P b/c ENSG00000177233
NOTE: ASS1P10 is replacing ASS1P10 b/c ENSG00000215325
NOTE: ANKRD62P1 is replacing ANKRD62P1 b/c ENSG00000259271
NOTE: RN7SL597P is replacing RN7SL597P b/c ENSG00000244264
NOTE: FBN2 is replacing FBN2 b/c ENSG00000138829
NOTE: KRT17 is replacing KRT17 b/c ENSG00000128422
NOTE: IRAK1BP1 is replac

...

NOTE: TAS2R40 is replacing TAS2R40 b/c ENSG00000221937
NOTE: MYL8P is replacing MYL8P b/c ENSG00000229596
NOTE: TUBB8P2 is replacing TUBB8P2 b/c ENSG00000218896
NOTE: SEPT14P19 is replacing SEPT14P19 b/c ENSG00000281379
NOTE: ATP2B2 is replacing ATP2B2 b/c ENSG00000157087
NOTE: GSTA6P is replacing GSTA6P b/c ENSG00000223622
NOTE: IGHVIII-47-1 is replacing IGHVIII-47-1 b/c ENSG00000253862
NOTE: DHX30 is replacing DHX30 b/c ENSG00000132153
NOTE: HIGD1AP11 is replacing HIGD1AP11 b/c ENSG00000227644
NOTE: NARF-AS1 is replacing RP13-991F5.2 b/c ENSG00000266445
NOTE: TTYH2 is replacing TTYH2 b/c ENSG00000141540
NOTE: DGCR8 is replacing DGCR8 b/c ENSG00000128191
NOTE: PPFIBP2 is replacing PPFIBP2 b/c ENSG00000166387
NOTE: BNIP3P18 is replacing BNIP3P18 b/c ENSG00000269055
NOTE: RN7SL552P is replacing RN7SL552P b/c ENSG00000239466
NOTE: IGKV1D-37 is replacing IGKV1D-37 b/c ENSG00000250036
NOTE: CTNNA3 is replacing CTNNA3 b/c ENSG00000183230
NOTE: RN7SL640P is replacing RN7SL640P b/c ENSG000002

NOTE: MIR638 is replacing MIR638 b/c ENSG00000207972
NOTE: SLC14A2 is replacing SLC14A2 b/c ENSG00000132874
NOTE: RPL17-C18orf32 is replacing RPL17-C18orf32 b/c ENSG00000215472
NOTE: DEFB108F is replacing RP11-1396O13.14 b/c ENSG00000188438
NOTE: RANP8 is replacing RANP8 b/c ENSG00000233524
NOTE: EIF4E1B is replacing EIF4E1B b/c ENSG00000175766
NOTE: RARRES2P11 is replacing RARRES2P11 b/c ENSG00000283123
NOTE: PCDHB17P is replacing PCDHB17P b/c ENSG00000255622
NOTE: VMP1 is replacing VMP1 b/c ENSG00000062716
NOTE: RNY3P4 is replacing RNY3P4 b/c ENSG00000207157
NOTE: UBL7 is replacing UBL7 b/c ENSG00000138629
NOTE: RPL36AP33 is replacing RPL36AP33 b/c ENSG00000235619
NOTE: CADM2-AS1 is replacing CADM2-AS1 b/c ENSG00000239519
NOTE: RNU6-65P is replacing RNU6-65P b/c ENSG00000252141
NOTE: HIF1AN is replacing HIF1AN b/c ENSG00000166135
NOTE: ANO10 is replacing ANO10 b/c ENSG00000160746
NOTE: SNORD118 is replacing SNORD118 b/c ENSG00000200463
NOTE: CLIC4P2 is replacing CLIC4P2 b/c ENSG00000

NOTE: KATNAL1 is replacing KATNAL1 b/c ENSG00000102781
NOTE: RPL7P35 is replacing RPL7P35 b/c ENSG00000233518
NOTE: AKAP2 is replacing AKAP2 b/c ENSG00000241978
NOTE: PLCD1 is replacing PLCD1 b/c ENSG00000187091
NOTE: LINC00605 is replacing LINC00605 b/c ENSG00000251533
NOTE: HIST1H2AJ is replacing HIST1H2AJ b/c ENSG00000276368
NOTE: LINC01680 is replacing RP11-309H21.2 b/c ENSG00000233882
NOTE: GLRXP2 is replacing RP11-750I4.2 b/c ENSG00000258544
NOTE: CHKA is replacing CHKA b/c ENSG00000110721
NOTE: NUCB2 is replacing NUCB2 b/c ENSG00000070081
NOTE: OTUD4P1 is replacing OTUD4P1 b/c ENSG00000118976
NOTE: MIR6511A4 is replacing MIR6511A4 b/c ENSG00000273613
NOTE: RPL36AP6 is replacing RPL36AP6 b/c ENSG00000231293
NOTE: FTLP15 is replacing FTLP15 b/c ENSG00000233952
NOTE: YRDCP3 is replacing YRDCP3 b/c ENSG00000230859
NOTE: MIR3663 is replacing MIR3663 b/c ENSG00000266782
NOTE: TBX2-AS1 is replacing TBX2-AS1 b/c ENSG00000267280
NOTE: FAM212A is replacing FAM212A b/c ENSG00000185614
NOTE

NOTE: SCOC is replacing SCOC b/c ENSG00000153130
NOTE: OR10J9P is replacing OR10J9P b/c ENSG00000249197
NOTE: CCDC185 is replacing CCDC185 b/c ENSG00000178395
NOTE: MAP3K11 is replacing MAP3K11 b/c ENSG00000173327
NOTE: NUP214 is replacing NUP214 b/c ENSG00000126883
NOTE: OR9I1 is replacing OR9I1 b/c ENSG00000172377
NOTE: KCNA1 is replacing KCNA1 b/c ENSG00000111262
NOTE: HMGN1P13 is replacing HMGN1P13 b/c ENSG00000249619
NOTE: RNU1-34P is replacing RNU1-34P b/c ENSG00000201291
NOTE: PCDHB1 is replacing PCDHB1 b/c ENSG00000171815
NOTE: SPINK1 is replacing SPINK1 b/c ENSG00000164266
NOTE: FAM157C is replacing FAM157C b/c ENSG00000260528
NOTE: HLA-V is replacing HLA-V b/c ENSG00000181126
NOTE: AP4M1 is replacing AP4M1 b/c ENSG00000221838
NOTE: PKHD1L1 is replacing PKHD1L1 b/c ENSG00000205038
NOTE: RNU6-762P is replacing RNU6-762P b/c ENSG00000252916
NOTE: IGLV3-4 is replacing IGLV3-4 b/c ENSG00000226420
NOTE: MTCO3P41 is replacing MTCO3P41 b/c ENSG00000237466
NOTE: EP400 is replacing EP4

NOTE: KIAA1211 is replacing KIAA1211 b/c ENSG00000109265
NOTE: KANSL1L is replacing KANSL1L b/c ENSG00000144445
NOTE: RNU6-384P is replacing RNU6-384P b/c ENSG00000201770
NOTE: MIR339 is replacing MIR339 b/c ENSG00000199023
NOTE: RN7SL123P is replacing RN7SL123P b/c ENSG00000266863
NOTE: TRMT5 is replacing TRMT5 b/c ENSG00000126814
NOTE: WDR70 is replacing WDR70 b/c ENSG00000082068
NOTE: RN7SL530P is replacing RN7SL530P b/c ENSG00000239910
NOTE: RNU6-710P is replacing RNU6-710P b/c ENSG00000222356
NOTE: RN7SL277P is replacing RN7SL277P b/c ENSG00000240490
NOTE: GLIPR1L2 is replacing GLIPR1L2 b/c ENSG00000180481
NOTE: RNA5SP173 is replacing RNA5SP173 b/c ENSG00000201727
NOTE: LINC01353 is replacing LINC01353 b/c ENSG00000231507
NOTE: TESK2 is replacing TESK2 b/c ENSG00000070759
NOTE: IPO9-AS1 is replacing IPO9-AS1 b/c ENSG00000231871
NOTE: FGF2 is replacing FGF2 b/c ENSG00000138685
NOTE: GLULP4 is replacing GLULP4 b/c ENSG00000178723
NOTE: TECRL is replacing TECRL b/c ENSG00000205678
NO

In [16]:
len(gene_tissue)

59171

In [17]:
sum(gene_tissue['ENSG00000002330'] * gene_tissue['ENSG00000003402'])

188

In [18]:
np.dot(gene_tissue['ENSG00000002330'], gene_tissue['ENSG00000003402'])

188

In [19]:
np.dot(gene_tissue['ENSG00000002330'], gene_tissue['ENSG00000003402'])/min(np.count_nonzero(gene_tissue['ENSG00000002330']),np.count_nonzero(gene_tissue['ENSG00000003402']))

0.73151750972762641

With this relative scoring scheme I am trying to normalize based only the reported measures
that is; an unreported measurement is not counted as penalty alowing more "perfect matches"  

however since this normalization is unique for each pair of genes  
it means comparing scores between different pairs is not as meaningful.


In [20]:
np.dot(gene_tissue['ENSG00000002330'], gene_tissue['ENSG00000003402'])/len(gene_tissue['ENSG00000003402'])

0.60256410256410253

this "absolute" normalizing does allow different pairs to be compared, but I expect vanishingly few will top out even if they agree on all available points.

Probably keep both, and use the relative score to surface interesting associations 
and the absolute to give people something they can compare. 

  !# Try to find NCBIGene_id for Ensenbl_ID
        # try to convert ids from ensembl to ncbi 
        gene_id = ensg_ncbi[gene_id] if gene_id in ensg_ncbi else gene_id

        !# last gasp conversion based on gene symbol
        if gene_id == ensg and gene_name in sym_ncbi:
            # print('WARN: ' + sym_ncbi[gene_name] + ' is replacing '+ ensg + ' b/c '+ gene_name)
            gene_id = sym_ncbi[gene_name]

In [21]:
# save a copy 
#with open('gene_tissue_curve.yaml', 'w') as fh:
#    yaml.dump(gene_tissue, fh)
# Ack! can't dump numpy arrays like this they are binary   

# gene_tissue.savetext('gene_tissue_curve.txt')
# nor like that because the container is not numpy 

In [22]:
genes = list(gene_tissue) # need a fixed order
genlen = len(genes)
gtlen = len(gene_tissue[genes[1]])
genes[1]

# note: thanks to the opaqueness of the datastructures 
# we have no idea how we are hopping around in memory
# while marching straight down this list.

'ENSG00000106153'

((59172^2)/2)-59172 == 1,750,603,620

1.75  billion comparisons involving pairs vectors with 300 values each

maybe I should switch to Julia for this next part.

#### Take 2
In retrospect (looking at first run which tooks about 2.5 hours),   
keeping only those for which there is no reported disagreement 
subjectivly results in too many edges. 
So the relative score is a constant 'one'
and the absolute score is more directly expressed as 
how many concordant mesurements or since we are only taking consensus results
how many tissues did this pair of genes up|down express in the same direction?

there is also a slight difference in how zeros are counted  
first run 'unreporteds' were counted for each gene-tissue-set insependently
then the one with the most zeros determined the relative limit.

For this second run I combine the vectors before counting zeros.
since this could result in fewer zeros than the min of the components
there is a risk of generating even more edges.  
will revert if it is a problem.

#### Take three

well changing the zero count was a problem
count/per 1000 went 500000 from to 700000 which is likely more fair
but time/per 1000 went from 5min to a half hour 


In [ ]:

rel_scores={}
print(str(datetime.datetime.now()))
for a in range(1,genlen-1):
    a_name = genes[a]
    if 0==a%1000:
        print(str(a) + ' kept ' + str(len(rel_scores)) +  " " + str(datetime.datetime.now()))      
    for b in range(a+1, genlen):
        dot_ab = np.dot(gene_tissue[a_name], gene_tissue[genes[b]])
        concordant = np.count_nonzero(np.bitwise_and(gene_tissue[genes[a]], gene_tissue[genes[b]]))
        # limit output
        if concordant == dot_ab:
            rel_scores[a_name + ',' + genes[b]] = concordant

print(str(genlen) + ' kept ' + str(len(rel_scores)) +  " " + str(datetime.datetime.now()))

2017-10-06 20:53:39.550808
1000 kept 672490 2017-10-06 20:59:27.843574
2000 kept 1317982 2017-10-06 21:05:09.001333
3000 kept 1933198 2017-10-06 21:10:47.335389
4000 kept 2681596 2017-10-06 21:16:17.870343
5000 kept 3306901 2017-10-06 21:21:42.274127
6000 kept 3951835 2017-10-06 21:27:04.367899
7000 kept 4599316 2017-10-06 21:32:22.645159
8000 kept 5135040 2017-10-06 21:37:33.393990
9000 kept 5653236 2017-10-06 21:42:37.119431
10000 kept 6157658 2017-10-06 21:47:33.240432
11000 kept 6648619 2017-10-06 21:52:23.142765
12000 kept 7237877 2017-10-06 21:57:06.586821
13000 kept 7655657 2017-10-06 22:01:45.832260
14000 kept 8097099 2017-10-06 22:06:13.819813
15000 kept 8562374 2017-10-06 22:10:33.531514
16000 kept 8980217 2017-10-06 22:14:51.949059
17000 kept 9364276 2017-10-06 22:18:55.844249
18000 kept 9886261 2017-10-06 22:22:55.103269
19000 kept 10344329 2017-10-06 22:26:45.888754


In [ ]:
len(rel_scores)/len(genes)


In [ ]:
associated = list(rel_scores.keys())

In [ ]:
with open('bgee_coexp_network.yaml', 'w') as fh:
    yaml.dump(associated, fh)

In [ ]:
list(associated[1].split(','))

In [ ]:
G = nx.Graph()
for edge in associated:
    ee=edge.split(',')
    G.add_edge(ee[0],ee[1])

In [ ]:
print(nx.number_connected_components(G))

__WOW__ exatctly one fewer cluster than the Human Protein Atlas notenook.


In [ ]:
clusters = nx.connected_components(G)

In [ ]:
c = next(clusters)

In [ ]:
for e in c:
    if e in ensg_sym:
        print(ensg_sym[e])
    else: 
        print(e)

In [ ]:
G.neighbors('ENSG00000255065')

In [ ]:
#nx.average_degree_connectivity(G)

In [ ]:
#nx.average_neighbor_degree(G)

keeping only those edges between genes (ENSMBLs)  
for which there is no contrary evidence of co-expression  
results in an awful lot of edges.  

it may be help clarity to filter genes that are ubiquoisly co expressed

In [ ]:
degree_sequence = sorted(G.degree().items(), key=operator.itemgetter(1),  reverse=True)
# sorted(x.items(), key=operator.itemgetter(1))

In [ ]:
# adapted from: 
# https://networkx.github.io/documentation/stable/auto_examples/drawing/plot_degree_rank.html#sphx-glr-auto-examples-drawing-plot-degree-rank-py
plt.plot([d for n,d in degree_sequence], 'b-', marker='o')
plt.title("Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")
plt.show()

In [ ]:
plt.loglog([d for n,d in degree_sequence], 'b-', marker='o')
plt.title("Log-Log Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")
plt.show()

That looks like the first ten are outregous   
then the first thousands (the flat line) very suspect

Setting a threshold on degree at 1,000 

I wonder if the piece wise nature of the curve reflects 
different dataset policy and procdures  




In [ ]:
#
with open('bgee_rank_degree.txt', 'w') as fh:
    fh.write(str([d for n,d in degree_sequence]))

In [ ]:
# ! tr ',' '\n' < bgee_rank_degree.txt  | tr -d "] ["  | head -200 | grep -n .
# G.nodes()[2].rank()
# G.degree().items()

In [ ]:
k=0
for n, r in  G.degree().items(): 
    if r > 1000:
        k+=1
        print(str(k) + ": removing " + n + " with " + str(r)  + " edges") 
        G.remove_node(n)

In [ ]:
plt.plot([d for n,d in sorted(G.degree().items(), key=operator.itemgetter(1),  reverse=True)], 'b-', marker='o')
plt.title("Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")
plt.show()

In [ ]:
np.mult([2,3,4], [5,7,9])